In [1]:
cd scripts/

/home/ubuntu/facebook/scripts


In [3]:
import grid_generation as grid
import numpy as np

In [18]:
g = grid.Grid(200, 50, 20, 5, pref = 'grid')

In [15]:
place_stats_dict = get_stat_dict(np.vstack(map(lambda f: load_data(f, (0, 1, 4, 5, 6)), place_file_names)))
#place_stats_dict[place_id] = [count, a_median, x_median, y_median]

time_dict = get_stat_dict(np.vstack(map(lambda f: load_data(f, (0, 1, 2, 3, 4, 5, 6, 7, 8)), time_file_names)))
#time_dict[place_id] = [hour, hour_2, hour_3, hour_4, hour_6, hour_8, weekday, month]

In [17]:
print len(place_stats_dict)
print len(time_dict)

108390
108390


In [84]:
def get_stat_dict(matrix):
    return dict(map(lambda x: [int(x[0]), x[1:]], matrix))

def load_data_from_grid(m, n):
    f = g.getGridFile(m, n)
    return np.loadtxt(f, delimiter = ',')

def get_time_stat(id):
    if id in time_dict:
        return time_dict[id]
    else:
        return [-999]*8

def get_place_stat(id):
    if id in place_stats_dict:
        return place_stats_dict[id]
    else:
        return [-999]*4
    
submission_name = 'ec2_colsample_bytree0_6_scale_pos_weight1_min_child_weight6_subsample0_9_eta0_1_alpha0_005_max_depth3_gamma0_1_th3_'
place_file_names = map(lambda x: '../split_train/' + str(x) + '_place', range(1, 23))
time_file_names = map(lambda x: '../split_train/' + str(x) + '_time', range(1, 23))
load_data = lambda file_name, use_cols: np.loadtxt(file_name, dtype = 'float', delimiter=',', skiprows=1, usecols = use_cols)

In [98]:

        
def get_features_from_ids(xyat, ids):
    return np.concatenate(map(lambda id: get_feature_from_id(xyat, id), ids))
    
def get_feature_from_id(xyat, id):
    f = get_time_stat(id)
    p = get_place_stat(id)
    abd = abs_dt(xyat[:2], p[-2:])
    return np.hstack((f, p, abd))
    
def abs_dt(p1, p2):
    return np.abs(p1[0] - p2[0]) + np.abs(p1[1] - p2[1]) 

def get_top_places_file(m, n):
    return g.getFolder()[:-1] + '_' + submission_name + '/' + '_'.join(['top_t_preds', str(m), str(n)]) + '.csv'

data = load_data_from_grid(0, 0)
top_t_data = np.loadtxt(get_top_places_file(0, 0), delimiter = ',', dtype = int)

data_dict = get_stat_dict(data)
top_t_dict = get_stat_dict(top_t_data)

features = {}
for i in data_dict.keys():
    if i in top_t_dict:
        features[i] = get_features_from_ids(data_dict[i], top_t_dict[i])
    else:
        features[i] = np.array([-999]*130)

feature_items = sorted(features.items(), cmp = lambda x, y: cmp(x[0], y[0]))
feature_data = np.array(map(lambda row: np.hstack(([row[0]], row[1])), feature_items))

In [125]:
import itertools
from multiprocessing import Pool
import numpy as np
#Import libraries:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier  #GBM algorithm
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search

import matplotlib.pylab as plt
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4
%matplotlib inline
from sklearn.cross_validation import train_test_split
import pickle

from sklearn.preprocessing import LabelEncoder

import xgboost as xgb

import traceback
import grid_generation as grid

In [99]:
def transform_x(X, x_transformer = None):
    """
    X = [[x, y, a, t]]
    """
    fw = [500., 1000., 4., 3., 2., 10., 10.]
    minute_v = X[:, 3]%60
    hour_v = X[:, 3]//60
    weekday_v = hour_v//24
    month_v = weekday_v//30
    year_v = (weekday_v//365 + 1)*fw[5]
    hour_v = ((hour_v%24 + 1) + minute_v/60.0)*fw[2]
    hour_v_2 = (X[:, 3]%(60*60*24))//(60*60*2)
    hour_v_3 = (X[:, 3]%(60*60*24))//(60*60*3)
    hour_v_4 = (X[:, 3]%(60*60*24))//(60*60*4)
    hour_v_6 = (X[:, 3]%(60*60*24))//(60*60*6)
    hour_v_8 = (X[:, 3]%(60*60*24))//(60*60*8)
    weekday_v = (weekday_v%7 + 1)*fw[3]
    month_v = (month_v%12 +1)*fw[4]
    accuracy_v = np.log10(X[:, 2])*fw[6]
    x_v = X[:, 0]*fw[0]
    y_v = X[:, 1]*fw[1]
    return np.hstack((x_v.reshape(-1, 1),
                     y_v.reshape(-1, 1),
                     accuracy_v.reshape(-1, 1),
                     hour_v.reshape(-1, 1),
                     hour_v_2.reshape(-1, 1),
                     hour_v_3.reshape(-1, 1),
                     hour_v_4.reshape(-1, 1),
                     hour_v_6.reshape(-1, 1),
                     hour_v_8.reshape(-1, 1),
                     weekday_v.reshape(-1, 1),
                     month_v.reshape(-1, 1),
                     year_v.reshape(-1, 1)))

def transform_y(y, y_transformer = None):
    """
    place_ids to encoded array
    """
    y = y.astype(int)
    if y_transformer == None:
        label_encoder = LabelEncoder()
        label_encoder.fit(y)
        y_transformer = {'encoder': label_encoder}
    new_y = y_transformer['encoder'].transform(y).reshape(-1, 1)
    return (new_y, y_transformer)

def map3eval(preds, dtrain):
    actual = dtrain.get_label()
    predicted = preds.argsort(axis=1)[:,-np.arange(1,4)]
    metric = 0.
    for i in range(3):
        metric += np.sum(actual==predicted[:,i])/(i+1)
    metric /= actual.shape[0]
    return 'MAP@3', metric

In [100]:
g.generateCardinalityMatrix()

In [93]:
try:
    d = np.loadtxt('random')
except IOError:
    print "file doesnt exists"
    d = np.array([])

file doesnt exists


In [109]:

mask = np.array(map(lambda x: g.M[0][0][x] > 3, data[:, 5]))
train = data[mask, :]
masked_feat = feature_data[mask, :]

In [104]:
X = transform_x(train[:, (1, 2, 3, 4)])
y, enc = transform_y(train[:, 5])

In [110]:
tX = np.hstack((X, masked_feat[:, 1:]))

In [106]:
X.shape

(2645, 12)

In [111]:
tX.shape

(2645, 142)

In [114]:
dtrain = xgb.DMatrix(tX, label=np.ravel(y))

In [116]:
orig_params = {
    'num_class': len(enc['encoder'].classes_),
            'silent': 0,
            'nthread': 8,
            'eta': 0.1,
            'objective': 'multi:softprob',
            'max_depth': 8,
            'min_child_weight': 5,
            'gamma': 0.32,
            'subsample': 0.9,
            'colsample_bytree': 0.7,
            'scale_pos_weight': 1
            }

In [123]:
bst = xgb.train(orig_params, dtrain, 30, feval = map3eval, evals = [(dtrain, 'train')],verbose_eval = True)

[0]	train-MAP@3:0.824575
[1]	train-MAP@3:0.854442
[2]	train-MAP@3:0.883932
[3]	train-MAP@3:0.889603
[4]	train-MAP@3:0.897543
[5]	train-MAP@3:0.898299
[6]	train-MAP@3:0.903214
[7]	train-MAP@3:0.904348
[8]	train-MAP@3:0.908507
[9]	train-MAP@3:0.911531
[10]	train-MAP@3:0.914178
[11]	train-MAP@3:0.91569
[12]	train-MAP@3:0.916824
[13]	train-MAP@3:0.91758
[14]	train-MAP@3:0.918336
[15]	train-MAP@3:0.919471
[16]	train-MAP@3:0.920983
[17]	train-MAP@3:0.921739
[18]	train-MAP@3:0.922873
[19]	train-MAP@3:0.923629
[20]	train-MAP@3:0.924764
[21]	train-MAP@3:0.925898
[22]	train-MAP@3:0.925898
[23]	train-MAP@3:0.926276
[24]	train-MAP@3:0.926654
[25]	train-MAP@3:0.927032
[26]	train-MAP@3:0.927788
[27]	train-MAP@3:0.927788
[28]	train-MAP@3:0.928544
[29]	train-MAP@3:0.929301


In [130]:
dtrain1 = xgb.DMatrix(X, label = np.ravel(y))

In [132]:
bst = xgb.train(orig_params, dtrain1, 30, feval = map3eval, evals = [(dtrain1, 'train')],verbose_eval = True)

[0]	train-MAP@3:0.589414
[1]	train-MAP@3:0.663138
[2]	train-MAP@3:0.713422
[3]	train-MAP@3:0.731191
[4]	train-MAP@3:0.750473
[5]	train-MAP@3:0.765595
[6]	train-MAP@3:0.77656
[7]	train-MAP@3:0.781853
[8]	train-MAP@3:0.787146
[9]	train-MAP@3:0.796597
[10]	train-MAP@3:0.802268
[11]	train-MAP@3:0.80794
[12]	train-MAP@3:0.813611
[13]	train-MAP@3:0.818904
[14]	train-MAP@3:0.824197
[15]	train-MAP@3:0.827977
[16]	train-MAP@3:0.830246
[17]	train-MAP@3:0.834405
[18]	train-MAP@3:0.835917
[19]	train-MAP@3:0.839319
[20]	train-MAP@3:0.840832
[21]	train-MAP@3:0.845369
[22]	train-MAP@3:0.849149
[23]	train-MAP@3:0.853686
[24]	train-MAP@3:0.855577
[25]	train-MAP@3:0.858979
[26]	train-MAP@3:0.859735
[27]	train-MAP@3:0.86276
[28]	train-MAP@3:0.865028
[29]	train-MAP@3:0.868053


In [133]:
cv1 = xgb.cv({'num_class': len(enc['encoder'].classes_),
                 'silent': 1,
            'nthread': 8,
            'eta': 0.1,
            'objective': 'multi:softprob',
            'max_depth': 8,
            'min_child_weight': 5,
            'gamma': 0.32,
            'subsample': 0.9,
            'colsample_bytree': 0.7,
            'scale_pos_weight': 1}, 
                dtrain, 
                feval = map3eval, maximize = True, verbose_eval = True
        )

[0]	train-MAP@3:0.767499+0.00764725	test-MAP@3:0.755959+0.0149151
[1]	train-MAP@3:0.816875+0.00335238	test-MAP@3:0.802497+0.0145654
[2]	train-MAP@3:0.85036+0.00965743	test-MAP@3:0.836171+0.017396
[3]	train-MAP@3:0.874574+0.00379283	test-MAP@3:0.859629+0.0153501
[4]	train-MAP@3:0.879493+0.00701751	test-MAP@3:0.864926+0.0106883
[5]	train-MAP@3:0.88933+0.00490422	test-MAP@3:0.875899+0.0117352
[6]	train-MAP@3:0.892736+0.0044204	test-MAP@3:0.880817+0.0109267
[7]	train-MAP@3:0.895384+0.00491127	test-MAP@3:0.881952+0.010025
[8]	train-MAP@3:0.897465+0.00512539	test-MAP@3:0.884601+0.00861117
[9]	train-MAP@3:0.899357+0.00566282	test-MAP@3:0.885358+0.00823736


In [134]:
cv2 = xgb.cv({'num_class': len(enc['encoder'].classes_),
                 'silent': 1,
            'nthread': 8,
            'eta': 0.1,
            'objective': 'multi:softprob',
            'max_depth': 8,
            'min_child_weight': 5,
            'gamma': 0.32,
            'subsample': 0.9,
            'colsample_bytree': 0.7,
            'scale_pos_weight': 1}, 
                dtrain1, 
                feval = map3eval, maximize = True, verbose_eval = True
        )

[0]	train-MAP@3:0.59667+0.00440458	test-MAP@3:0.548997+0.0126506
[1]	train-MAP@3:0.653424+0.0126086	test-MAP@3:0.605751+0.012651
[2]	train-MAP@3:0.689557+0.00656983	test-MAP@3:0.645479+0.00928344
[3]	train-MAP@3:0.714151+0.00420484	test-MAP@3:0.660613+0.00884119
[4]	train-MAP@3:0.730799+0.00301503	test-MAP@3:0.674612+0.00788209
[5]	train-MAP@3:0.743473+0.00349852	test-MAP@3:0.681801+0.00417922
[6]	train-MAP@3:0.752175+0.00263495	test-MAP@3:0.686719+0.00893746
[7]	train-MAP@3:0.76031+0.0020896	test-MAP@3:0.692395+0.00723841
[8]	train-MAP@3:0.770337+0.00279342	test-MAP@3:0.701097+0.00475603
[9]	train-MAP@3:0.779418+0.00192913	test-MAP@3:0.707529+0.005104


In [129]:
sorted(fs.items(), cmp = lambda x, y: cmp(x[1], y[1]), reverse = True)

[('f24', 172),
 ('f23', 166),
 ('f12', 161),
 ('f20', 160),
 ('f22', 159),
 ('f0', 149),
 ('f1', 146),
 ('f36', 126),
 ('f3', 107),
 ('f35', 106),
 ('f37', 101),
 ('f25', 100),
 ('f33', 100),
 ('f76', 92),
 ('f2', 85),
 ('f63', 84),
 ('f21', 81),
 ('f50', 80),
 ('f19', 69),
 ('f34', 66),
 ('f128', 65),
 ('f112', 63),
 ('f13', 63),
 ('f89', 59),
 ('f14', 57),
 ('f88', 54),
 ('f47', 48),
 ('f102', 47),
 ('f140', 47),
 ('f32', 44),
 ('f111', 43),
 ('f115', 40),
 ('f124', 40),
 ('f62', 40),
 ('f141', 38),
 ('f64', 38),
 ('f138', 35),
 ('f77', 35),
 ('f116', 34),
 ('f114', 33),
 ('f51', 33),
 ('f26', 33),
 ('f75', 33),
 ('f38', 32),
 ('f10', 32),
 ('f48', 31),
 ('f113', 31),
 ('f100', 30),
 ('f60', 30),
 ('f49', 29),
 ('f127', 28),
 ('f129', 28),
 ('f9', 28),
 ('f66', 28),
 ('f72', 28),
 ('f59', 27),
 ('f98', 27),
 ('f103', 26),
 ('f139', 25),
 ('f137', 25),
 ('f27', 24),
 ('f99', 24),
 ('f11', 24),
 ('f86', 23),
 ('f45', 22),
 ('f46', 22),
 ('f109', 22),
 ('f101', 22),
 ('f85', 22),
 ('f18

In [144]:
sX = X[:100, :]
xY = y[:100]
strain = xgb.DMatrix(sX, label = np.ravel(xY))

In [145]:
bst = xgb.train(orig_params, strain, 30, feval = map3eval, evals = [(strain, 'train')],verbose_eval = True)

[0]	train-MAP@3:0
[1]	train-MAP@3:0
[2]	train-MAP@3:0
[3]	train-MAP@3:0
[4]	train-MAP@3:0
[5]	train-MAP@3:0
[6]	train-MAP@3:0
[7]	train-MAP@3:0
[8]	train-MAP@3:0
[9]	train-MAP@3:0
[10]	train-MAP@3:0
[11]	train-MAP@3:0
[12]	train-MAP@3:0
[13]	train-MAP@3:0
[14]	train-MAP@3:0
[15]	train-MAP@3:0
[16]	train-MAP@3:0
[17]	train-MAP@3:0
[18]	train-MAP@3:0
[19]	train-MAP@3:0
[20]	train-MAP@3:0
[21]	train-MAP@3:0
[22]	train-MAP@3:0
[23]	train-MAP@3:0
[24]	train-MAP@3:0
[25]	train-MAP@3:0
[26]	train-MAP@3:0
[27]	train-MAP@3:0
[28]	train-MAP@3:0
[29]	train-MAP@3:0
